In [1]:
import sys
sys.path.append('../modules')
import likelihood_predictor
from likelihood_predictor import PlastPredictor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import zscore
import pickle
from sklearn.feature_selection import SelectFromModel

In [14]:
pl_full = pd.read_pickle('../database/plasticizer_data_v10_polarity.pkl')
pl_pol = pd.concat([pl_full[pl_full.columns[1:195]], pl_full['Polarity']], axis=1)
all_cols = pl_pol.columns.to_numpy()
pl_data = pl_pol[all_cols].to_numpy()
lin_data = pd.read_pickle('../database/linolein_test.pkl')
lin_data['Polarity'] = 0.048856
lin_data = lin_data[all_cols].to_numpy()
org_full = pd.read_pickle('../database/org_v3.pkl')

In [15]:
psim1 = open("pubs_similarity.txt", 'r')
psim11 = [line.rstrip('\n') for line in psim1]
psim2 = open("pubs_othersim.txt", 'r')
psim22 = [line.rstrip('\n') for line in psim2]

In [16]:
org_full
# org_full['Dsim'] = psim11
# org_full['Nasim'] = psim22
org_full = org_full.sort_values(by ='DEHP_sim')

In [17]:
org_full

,SMILES,MaxEStateIndex,MinEStateIndex,qed,MolWt,NumValenceElectrons,NumRadicalElectrons,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,...,2045,2046,2047,Num Carbon,Num Polar,Polarity,Dsim,Nasim,DEHP_sim,Sodium_polynaphthalenesulfonate_sim
26359,CC1(COC2=C1C=C(C=C2C(C)(C)C)C(=O)CCCC3CC3)C,2.194444,0.000000,0.304180,112.625,34.0,0.0,0.096899,-1.000000,1.000000,...,0.0,0.0,0.0,4.0,2.0,0.056313,0.0,0.0029154518950437317,0.0,0.0029154518950437317
12355,COC1=CC(=C(C=C1C(=O)NCC2CCN(CC2)CCCN3C=CN=N3)Cl)N,6.875000,0.000000,0.288361,80.103,16.0,0.0,2.000000,-0.785891,1.250000,...,0.0,0.0,0.0,2.0,3.0,0.120155,0.0,0.0029239766081871343,0.0,0.0029239766081871343
33543,CC1=CC=C(C=C1)OS(=O)(=O)O,4.125000,0.916667,0.320752,96.201,20.0,0.0,-0.506777,-0.746611,1.333333,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
26096,CCC1=CC=CC=C1OCC(CNC2CCCC3=CC=CC=C23)O.C(=O)(C...,6.750000,0.000000,0.138349,81.119,16.0,0.0,1.000000,-0.372636,1.250000,...,0.0,0.0,0.0,0.0,5.0,0.000000,0.0,0.005865102639296188,0.0,0.005865102639296188
36692,CCC=CCC1=C(CCC1=O)C,9.555556,-0.611111,0.424467,380.732,89.0,3.0,3.000000,-1.000000,0.692308,...,0.0,0.0,0.0,7.0,4.0,0.217561,0.0,0.011594202898550725,0.0,0.011594202898550725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51834,CC1CC(CCC1N)CC2CCC(C(C2)C)N,11.781419,-1.118187,0.605487,264.321,104.0,0.0,0.338506,-0.477620,1.157895,...,0.0,0.0,1.0,7.0,2.0,0.075520,0.9090909090909091,0.12191103789126853,0.9090909090909091,0.12191103789126853
47955,C1=CC(=CC=C1C(=O)C2=CC=C(C=C2)NC3=CC=C(C=C3)S(...,11.992687,-1.179584,0.680217,308.330,120.0,0.0,0.338516,-0.481230,1.136364,...,0.0,0.0,1.0,7.0,3.0,0.132141,0.9273182957393483,0.13230769230769232,0.9273182957393483,0.13230769230769232
52690,C(C(=O)N)N(CC(=O)N)CC(=O)N,11.975968,-1.160953,0.744975,292.331,114.0,0.0,0.338516,-0.477620,1.190476,...,0.0,0.0,1.0,7.0,3.0,0.125285,0.9296482412060302,0.13425925925925927,0.9296482412060302,0.13425925925925927
45427,CC(=O)C1=CC=C(C=C1)NC(=O)N(C)N=O,11.993522,-1.149355,0.720163,294.347,116.0,0.0,0.338516,-0.477620,1.190476,...,0.0,0.0,1.0,8.0,3.0,0.110380,0.9585492227979274,0.12441679626749612,0.9585492227979274,0.12441679626749612


In [23]:
org_full['Sodium_polynaphthalenesulfonate_sim'].to_list()[20000]

'0.15553869499241274'

In [6]:
org_full = org_full[:15000]
org_data = org_full[all_cols].to_numpy()

In [7]:
reg_param = 10
pp = PlastPredictor(reg_param)
pp_model = pp.fit_model(pl_data, org_data)


In [8]:
cc=pp.clf.coef_

In [9]:
np.count_nonzero(cc)

124

In [10]:
clf_file = 'clf_oneiteration.pkl'
scaler_file = 'scaler_oneiteration.pkl'
pp.save_model(clf_file, scaler_file)

In [11]:
org_full2 = pd.read_pickle('../database/org_v3.pkl')
org_data2 = org_full2[all_cols].to_numpy()

In [12]:
org_acc = pp.predict(org_data2, type='binary', class_id='neg')
pl_acc = pp.predict(pl_data, type='binary', class_id='pos')

In [13]:
org_acc, pl_acc

(0.9590576156591056, 0.9571428571428572)